<a href="https://colab.research.google.com/github/Aggraj/Deep-Learning-CS-6910/blob/main/WandB_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from math import exp
from random import seed
from random import random
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation
 
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']
 
# Train a network for a fixed number of epochs
#def train_network(network, train, l_rate, n_epoch, n_outputs):
#	for epoch in range(n_epoch):
#		sum_error = 0
#		for row in train:
#			outputs = forward_propagate(network, row)
#			expected = [0 for i in range(n_outputs)]
#			expected[row[-1]] = 1
#			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
#			backward_propagate_error(network, expected)
#			update_weights(network, row, l_rate)
#		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
 
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row)
      expected = [0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
  return sum_error
    

    #return sum_error 
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
  print(layer)
  

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.0026279652850863837}]
[{'weights

In [95]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [10, 20]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3]
        },
    }
}


In [96]:
# The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 2,
        'batch_size': 128,
        'weight_decay': 0.0005,
        'learning_rate': 1e-3,
        'activation': 'relu',
        'optimizer': 'nadam',
        'seed': 42
    }

    # Initialize a new wandb run
    wandb.init(project='Assignment 1', entity='chaxin',config=config_defaults)
    
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    learning_rate = config.learning_rate
    epochs = config.epochs
    # Model training here
    loss=train_network(network, dataset, learning_rate, epochs, n_outputs)

#train_network(network, dataset, config.learning_rate, config.epochs, n_outputs)




# 3. Log metrics over time to visualize performance
    wandb.log({"loss": loss})

In [97]:
sweep_id = wandb.sweep(sweep_config, entity="chaxin", project="Assignment 1")

Create sweep with ID: 1t8ceyed
Sweep URL: https://wandb.ai/chaxin/Assignment%201/sweeps/1t8ceyed


In [98]:
wandb.agent(sweep_id, train,count=5)

wandb: Agent Starting Run: j6sn2b4z with config:
wandb: 	epochs: 20
wandb: 	learning_rate: 0.001
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


>epoch=0, lrate=0.001, error=1.058
>epoch=1, lrate=0.001, error=1.058
>epoch=2, lrate=0.001, error=1.057
>epoch=3, lrate=0.001, error=1.057
>epoch=4, lrate=0.001, error=1.057
>epoch=5, lrate=0.001, error=1.057
>epoch=6, lrate=0.001, error=1.056
>epoch=7, lrate=0.001, error=1.056
>epoch=8, lrate=0.001, error=1.056
>epoch=9, lrate=0.001, error=1.055
>epoch=10, lrate=0.001, error=1.055
>epoch=11, lrate=0.001, error=1.055
>epoch=12, lrate=0.001, error=1.055
>epoch=13, lrate=0.001, error=1.054
>epoch=14, lrate=0.001, error=1.054
>epoch=15, lrate=0.001, error=1.054
>epoch=16, lrate=0.001, error=1.053
>epoch=17, lrate=0.001, error=1.053
>epoch=18, lrate=0.001, error=1.053
>epoch=19, lrate=0.001, error=1.053


loss,1.05256
_runtime,3
_timestamp,1615320459
_step,0


loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: a32thewu with config:
wandb: 	epochs: 10
wandb: 	learning_rate: 0.01
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


...

loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: tn92vj6k with config:
wandb: 	epochs: 10
wandb: 	learning_rate: 0.01
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


>epoch=0, lrate=0.010, error=0.968
>epoch=1, lrate=0.010, error=0.967
>epoch=2, lrate=0.010, error=0.965
>epoch=3, lrate=0.010, error=0.963
>epoch=4, lrate=0.010, error=0.962
>epoch=5, lrate=0.010, error=0.960
>epoch=6, lrate=0.010, error=0.958
>epoch=7, lrate=0.010, error=0.957
>epoch=8, lrate=0.010, error=0.955
>epoch=9, lrate=0.010, error=0.954


loss,0.95357
_runtime,3
_timestamp,1615320503
_step,0


loss,▁
_runtime,▁
_timestamp,▁
_step,▁


20

NameError: ignored